# RFM Analysis | Python | Customer Segmentation

# 1. Import libraries, dataset

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv(r'C:/Users/Suresh/Downloads/archive/data.csv')
df.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/2010 8:26,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/2010 8:26,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,12/1/2010 8:28,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,12/1/2010 8:28,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/2010 8:34,1.69,13047.0,United Kingdom


# 2. Cleaning and Preprocessing

In [3]:
# removing na values and duplicates
df = df.dropna()
df = df.drop_duplicates()
# converting InvoiceDate column to pandas datetime format
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
# sorting records by InvoiceDate in ascending order
df = df.sort_values(by='InvoiceDate')
# removing trailing '.0' from CustomerID
df['CustomerID'] = df['CustomerID'].apply(lambda x: str(x)[:-2])
# add column Amount = quantity * unit_price
df['Amount'] = round(df['Quantity']*df['UnitPrice'],2)

# 3. Grouping and Aggregation

In [4]:
# taking only necessary columns and aggregating by 'CustomerID'
df = df[['CustomerID','InvoiceDate','Amount']]
rfm_df = df.groupby('CustomerID').agg(list).reset_index()
rfm_df.head(5)

,CustomerID,InvoiceDate,Amount
0,12346,"[2011-01-18 10:01:00, 2011-01-18 10:17:00]","[77183.6, -77183.6]"
1,12347,"[2010-12-07 14:57:00, 2010-12-07 14:57:00, 201...","[22.5, 37.5, 17.4, 19.8, 25.5, 15.0, 22.5, 19...."
2,12348,"[2010-12-16 19:09:00, 2010-12-16 19:09:00, 201...","[104.4, 240.0, 50.4, 13.2, 50.4, 13.2, 69.6, 4..."
3,12349,"[2011-11-21 09:51:00, 2011-11-21 09:51:00, 201...","[19.5, 10.5, 27.04, 12.48, 17.4, 17.85, 13.2, ..."
4,12350,"[2011-02-02 16:01:00, 2011-02-02 16:01:00, 201...","[15.0, 25.2, 15.0, 15.0, 25.2, 40.0, 17.4, 19...."


# 4. Find Recency, Frequency & Monetary Value

In [5]:
# find last dates for every customer
rfm_df['last_dates'] = rfm_df['InvoiceDate'].apply(lambda x: x[-1])
# find latest date of purchase
latest_date = df['InvoiceDate'].max()
# find recency by subtracting last_dates from maximum date
rfm_df['recency'] = rfm_df['last_dates'].apply(lambda x: (latest_date- x).days)

In [6]:
# find frequency by counting no. of transactions per customer
rfm_df['frequency'] = rfm_df['Amount'].apply(len)
# find monetary value by summing up transaction amounts per customer
rfm_df['monetary_value'] = rfm_df['Amount'].apply(sum)
# take only necessary columns
rfm_df=rfm_df[['CustomerID','recency','frequency','monetary_value']]

In [7]:
# binning recency col
rfm_df['r'] = pd.qcut(rfm_df['recency'].rank(method='first'),5,labels=[5,4,3,2,1]).tolist()
# binning frequency col
rfm_df['f'] = pd.qcut(rfm_df['frequency'].rank(method='first'),5,labels=[1,2,3,4,5]).tolist()
# binning monetary_value col
rfm_df['m'] = pd.qcut(rfm_df['monetary_value'].rank(method='first'),5,labels=[1,2,3,4,5]).tolist()
# concatenating r,f,m scores together
rfm_df['rfm_score'] = rfm_df['r'].apply(str) + rfm_df['f'].apply(str) + rfm_df['m'].apply(str)

# 5. Find Categories of customers

In [8]:
##          segment_names Recency_seg Freq_seg Monetary_seg
## 1            Champions       [4,5]    [4,5]        [4,5]
## 2       Loyal Accounts       [3,5]    [3,5]        [3,5]
## 3   Potential Loyalist       [3,5]    [2,3]        [2,4]
## 4            Promising       [4,5]    [1,2]        [3,5]
## 5       Need Attention       [2,3]    [1,2]        [4,5]
## 6       About To Sleep       [2,3]    [1,2]        [1,3]
## 7              At Risk       [1,2]    [1,5]        [3,5]
## 8                 Lost       [1,2]    [1,5]        [1,2]
## 9         Low spenders       [3,5]    [1,5]        [1,2]

In [9]:
def find_segments(df, customer_id):
    classes = []
    for row in df.iterrows():
        if (row[1]['r'] in [4,5]) & (row[1]['f'] in [4,5]) & (row[1]['m'] in [4,5]):
            classes.append({row[1][customer_id]:'Champions'})
        elif (row[1]['r'] in [4,5]) & (row[1]['f'] in [1,2]) & (row[1]['m'] in [3,4,5]):
            classes.append({row[1][customer_id]:'Promising'})
        elif (row[1]['r'] in [3,4,5]) & (row[1]['f'] in [3,4,5]) & (row[1]['m'] in [3,4,5]):
            classes.append({row[1][customer_id]:'Loyal Accounts'})
        elif (row[1]['r'] in [3,4,5]) & (row[1]['f'] in [2,3]) & (row[1]['m'] in [2,3,4]):
            classes.append({row[1][customer_id]:'Potential Loyalist'})
        elif (row[1]['r'] in [3,4,5]) & (row[1]['f'] in [1,2,3,4,5]) & (row[1]['m'] in [1,2]):
            classes.append({row[1][customer_id]:'Low Spenders'})
        elif (row[1]['r'] in [2,3]) & (row[1]['f'] in [1,2]) & (row[1]['m'] in [4,5]):
            classes.append({row[1][customer_id]:'Need Attention'})
        elif (row[1]['r'] in [2,3]) & (row[1]['f'] in [1,2]) & (row[1]['m'] in [1,2,3]):
            classes.append({row[1][customer_id]:"About to Sleep"})
        elif (row[1]['r'] in [1,2]) & (row[1]['f'] in [1,2,3,4,5]) & (row[1]['m'] in [3,4,5]):
            classes.append({row[1][customer_id]:'At Risk'})
        elif (row[1]['r'] in [1,2]) & (row[1]['f'] in [1,2,3,4,5]) & (row[1]['m'] in [1,2]):
            classes.append({row[1][customer_id]:"Lost"})
        else:
            classes.append({0:[row[1]['r'],row[1]['f'],row[1]['m']]})
    accs = [list(i.keys())[0] for i in classes]
    segments = [list(i.values())[0] for i in classes]
    df['segment'] = df[customer_id].map(dict(zip(accs,segments)))
    return df

In [10]:
rfm_df = find_segments(rfm_df, 'CustomerID')
rfm_df.head(10)

,CustomerID,recency,frequency,monetary_value,r,f,m,rfm_score,segment
0,12346,325,2,0.00,1,1,1,111,Lost
1,12347,1,182,4310.00,5,5,5,555,Champions
2,12348,74,31,1797.24,2,3,4,234,At Risk
3,12349,18,73,1757.55,4,4,4,444,Champions
4,12350,309,17,334.40,1,2,2,122,Lost
5,12352,35,95,1545.41,3,4,4,344,Loyal Accounts
6,12353,203,4,89.00,1,1,1,111,Lost
7,12354,231,58,1079.40,1,3,4,134,At Risk
8,12355,213,13,459.40,1,1,2,112,Lost
9,12356,22,59,2811.43,4,4,5,445,Champions


# The End